In [1]:
import NN_Module as nnm
import torch
import numpy as np
import numpy.ma as ma
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import math
import time
import pandas as pd
from pandas import read_csv
import argparse
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
out_file = '3169_0'

In [3]:
names = {}
names['cQei'] = 0
names['cQl3i'] = 1
names['cQlMi'] = 2
names['cbW'] = 3
names['cpQ3'] = 4
names['cpQM'] = 5
names['cpt'] = 6
names['cptb'] = 7
names['ctG'] = 8
names['ctW'] = 9
names['ctZ'] = 10
names['ctei'] = 11
names['ctlSi'] = 12
names['ctlTi'] = 13
names['ctli'] = 14
names['ctp'] = 15

In [4]:
save_dict = torch.load(f'./{out_file}_model+.pt')
best_model_state = save_dict['model']
parameters_save = save_dict['parameters']
input_stats = save_dict['input_stats']
output_stats = save_dict['output_stats']

In [5]:
model = nnm.create_model(152, 1, parameters_save) # Hard-coded for the inclusion of square and cross terms
model.load_state_dict(best_model_state)
model.eval()

Sequential(
  (0): Linear(in_features=152, out_features=700, bias=True)
  (1): ReLU()
  (2): Linear(in_features=700, out_features=700, bias=True)
  (3): ReLU()
  (4): Linear(in_features=700, out_features=700, bias=True)
  (5): ReLU()
  (6): Linear(in_features=700, out_features=1, bias=True)
)

In [6]:
start = torch.tensor([[1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]], requires_grad=True)

In [7]:
optimizer = torch.optim.Adam([start],lr=2e-0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=5, threshold=1e-6)

In [8]:
# Functions
def evaluate(inputs):
    # Assumes model uses square and cross terms as well as standardization
    num_inputs = inputs.shape[0]
    inputs_all = torch.zeros((num_inputs, 152)).cuda()
    squares = inputs**2
    cross_terms = torch.zeros((num_inputs, 120)).cuda()
    idx = 0
    for i in range(16):
        for j in range(i):
            cross_terms[:,idx] = inputs[:,i] * inputs[:,j]
            idx += 1
    inputs_all[:,0:16] = inputs
    inputs_all[:,16:32] = squares
    inputs_all[:,32:152] = cross_terms
    std_inputs = nnm.affine_transform(inputs_all, input_stats)
    std_outputs = model(std_inputs)
    outputs = nnm.affine_untransform(std_outputs, output_stats)
    
    return outputs

In [9]:
epochs = 100
graph_data = {}
graph_data['NLL_epochs'] = np.zeros(epochs)
graph_data['NLL_vals'] = np.zeros(epochs)

In [10]:
torch.autograd.set_detect_anomaly(True)
for epoch in range(epochs):
    output = evaluate(start)
    output_cp = output.detach().clone()
    graph_data['NLL_vals'][epoch] = output_cp.cpu().detach().numpy().flatten()
    graph_data['NLL_epochs'][epoch] = epoch
    optimizer.zero_grad()
    output.backward()
    #start.grad[:,0] = 0
    optimizer.step()
    #scheduler.step(output)

In [11]:
graph_data['NLL_vals']

array([ 1.10976124e+01,  7.64644241e+00,  3.94184265e+01,  1.63834686e+01,
        4.85020065e+00,  9.73076630e+00,  2.14212418e+00,  9.68619347e+00,
        1.23299522e+01,  5.03292656e+00,  1.92435074e+00,  5.52701378e+00,
        6.70057297e+00,  4.83410835e+00,  3.46294785e+00,  3.60908127e+00,
        2.55877304e+00,  2.22789764e+00,  3.24007034e+00,  2.92007446e+00,
        9.24732208e-01,  1.49308395e+00,  2.85624313e+00,  2.61367798e+00,
        1.30539322e+00,  6.57226562e-01,  1.11139679e+00,  1.65716171e+00,
        1.26962280e+00,  8.00430298e-01,  8.86554718e-01,  1.08928680e+00,
        1.10309982e+00,  7.19852448e-01,  5.69137573e-01,  6.17141724e-01,
        5.76030731e-01,  4.23137665e-01,  4.53926086e-01,  5.88520050e-01,
        5.89317322e-01,  3.86398315e-01,  2.81009674e-01,  3.73775482e-01,
        4.26277161e-01,  3.54690552e-01,  2.74620056e-01,  3.05622101e-01,
        3.34320068e-01,  2.41901398e-01,  1.94168091e-01,  2.00370789e-01,
        2.14706421e-01,  

In [12]:
start

tensor([[-2.6467e-02, -4.4335e+00,  4.6640e-01, -2.8668e+00, -1.5853e+00,
          4.9553e-01, -3.6579e+00, -2.0063e-02, -7.5962e-01, -1.1298e+00,
         -1.2296e+00,  2.9958e-01, -1.2131e-02, -6.3080e-03,  2.8739e-01,
         -6.7580e+00]], requires_grad=True)

In [39]:
to_evaluate = torch.tensor([[ 6.0213e-02,  3.3749e+00,  3.7628e-01,  2.8397e+00, -1.6448e+00,
         4.3661e-01, -3.0663e-01,  1.2550e-01, -7.8860e-01,  4.9036e-01,
         6.3967e-01,  3.2782e-01, -2.9341e-02,  4.3780e-03,  1.7415e-01,
         2.5197e+01], [ 1.6234e-01, -4.2915e+00,  5.4159e-01,  2.8956e+00, -1.8258e+00,
         6.0013e-01, -3.5231e-01, -1.8007e-01, -7.9362e-01,  4.8166e-01,
         5.9329e-01,  3.6793e-01,  5.3760e-02, -1.1208e-02,  1.9321e-01,
         2.4757e+01], [ 0.0669,  3.5380,  0.4758,  2.8782, -1.9349,  0.9732, -0.2907, -0.0379,
        -0.8038,  0.6113,  0.7242,  0.3590, -0.1553,  0.0268,  0.2138, 24.4693], [ 1.2985e-01, -3.5192e+00,  4.1565e-01,  3.1828e+00, -1.9014e+00,
         4.3884e-01, -1.4247e+00,  8.0994e-03, -8.3554e-01,  3.7226e-01,
         4.7234e-01,  4.2801e-01, -9.0541e-03, -2.8858e-03,  2.2065e-01,
         2.3819e+01], [ 1.5048e-01,  3.3544e+00,  4.4847e-01,  2.8623e+00, -1.8159e+00,
         1.0667e+00, -2.5197e-01, -2.3042e-02, -7.9457e-01,  6.7596e-01,
         8.0256e-01,  3.0830e-01, -1.6220e-02,  2.6225e-02,  2.2623e-01,
         2.4561e+01], [ 1.4338e-01, -4.0218e+00,  4.8228e-01,  2.9098e+00, -1.5795e+00,
        -8.0291e-01, -2.1653e+00, -9.4478e-02, -7.9692e-01,  3.3659e-01,
         4.3358e-01,  3.0948e-01, -1.3669e-01,  3.5782e-03,  1.5387e-01,
         2.5035e+01], [ 1.6817e-01,  3.6427e+00,  6.2578e-01,  2.9871e+00, -1.6547e+00,
        -9.7204e-01, -3.4458e+00,  1.3628e-01, -8.0976e-01,  2.9929e-01,
         3.9542e-01,  3.2183e-01, -3.7706e-03, -4.9942e-03,  2.1415e-01,
         2.4272e+01], [ 1.0625e-01,  3.4198e+00,  4.9469e-01,  2.8693e+00, -1.7446e+00,
         5.4859e-01, -8.3345e-01,  1.1378e-01, -7.8866e-01,  5.8399e-01,
         7.9257e-01,  3.2221e-01, -2.7576e-02,  1.8747e-02,  1.9661e-01,
         2.5093e+01], [ 9.2890e-02,  3.1916e+00,  3.7979e-01,  2.9771e+00, -1.9056e+00,
         5.0856e-01, -1.3842e+00, -1.1487e-01, -8.4904e-01,  5.5529e-01,
         6.9341e-01,  3.1823e-01, -4.8362e-02,  1.7293e-02,  1.6860e-01,
         2.3297e+01], [ 5.5410e-02,  3.4005e+00,  5.6593e-01,  2.9673e+00, -1.7484e+00,
        -1.6854e-01, -1.3075e+00,  1.8398e-02, -8.0335e-01, -8.5344e-02,
        -8.2854e-02,  2.8581e-01,  3.0237e-02,  5.1308e-03,  1.8586e-01,
         2.5130e+01], [ 1.3622e-02, -4.0972e+00,  5.2002e-01,  3.0008e+00, -1.6982e+00,
        -7.3103e-01, -2.4295e+00,  3.0698e-01, -8.0364e-01,  9.1062e-03,
        -8.1047e-02,  3.2638e-01,  2.0288e-02, -1.3038e-02,  3.3264e-01,
         2.4814e+01], [ 3.5662e-01,  2.9557e+00,  5.1971e-01,  3.2421e+00, -2.0790e+00,
         7.2655e-02, -3.5235e+00, -9.0030e-02, -8.7726e-01,  5.7353e-01,
         6.9948e-01,  2.9282e-01, -9.2961e-02,  1.7405e-02,  2.1603e-01,
         2.1435e+01], [ 4.7732e-02, -3.8734e+00,  4.1129e-01,  2.8950e+00, -1.6996e+00,
         6.1793e-01, -8.2273e-01,  1.9653e-02, -8.1153e-01,  5.1119e-01,
         5.7374e-01,  3.9238e-01, -5.5718e-05, -7.4179e-03,  1.4864e-01,
         2.4166e+01], [ 9.0746e-02,  3.4030e+00,  4.7531e-01,  2.8950e+00, -1.8156e+00,
         9.9970e-01, -3.5344e-01,  4.9026e-02, -7.9755e-01,  6.0824e-01,
         7.7305e-01,  3.9806e-01, -1.0675e-01, -7.4945e-05,  1.2289e-01,
         2.4808e+01], [ 3.3037e-01,  3.6416e+00,  5.0886e-01,  2.8609e+00, -1.7479e+00,
         4.2149e-01, -9.9849e-01,  1.9761e-01, -8.0366e-01,  5.9773e-01,
         7.2790e-01,  4.7363e-01, -5.2533e-02,  1.0588e-02,  2.6655e-01,
         2.4581e+01], [ 5.6557e-03, -4.7137e+00,  4.9075e-01,  2.8822e+00, -1.6134e+00,
         2.1382e-01, -3.0181e+00,  1.4678e-01, -7.8400e-01, -9.4868e-01,
        -1.0582e+00,  2.3973e-01,  1.1341e-02,  1.0810e-02,  1.9459e-01,
        -6.9764e+00]])

In [40]:
evaluate(to_evaluate)

tensor([[-0.2341],
        [-0.2394],
        [-0.2448],
        [-0.2280],
        [-0.2431],
        [-0.2329],
        [-0.2151],
        [-0.2469],
        [-0.2382],
        [-0.2130],
        [-0.2232],
        [-0.1975],
        [-0.2392],
        [-0.2489],
        [-0.2458],
        [-0.0288]], device='cuda:0', grad_fn=<AddBackward0>)

In [31]:
to_evaluate = torch.tensor([[ 1.5059e-02, -9.9333e+00,  1.3617e+00,  1.5434e+00,  8.5782e-01,
        -1.4564e+00,  2.6855e-01, -3.3527e-01, -7.3721e-01,  2.4757e-01,
         2.2328e-01,  4.3129e-01, -2.5876e-02,  5.4869e-03,  2.7480e-03,
         2.7298e+01]])

In [32]:
evaluate(to_evaluate)

tensor([[2.2261]], device='cuda:0', grad_fn=<AddBackward0>)